In [8]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [9]:
def metrics(raw_img, new_img):
    result = []
    for c in range(0, 3):
        mse = np.mean(np.power((raw_img[5:-5, 5:-5, c] - new_img[5:-5, 5:-5, c]), 2))
        if mse == 0:
            result.append('perfect!!')
        else:
            result.append(10 * np.log10(255*255/mse))

    return result

def make_bayer(img):
    new_img = np.zeros_like(img)

    new_img[0::2, 0::2, 0] = img[0::2, 0::2, 0]
    new_img[0::2, 1::2, 1] = img[0::2, 1::2, 1]
    new_img[1::2, 0::2, 1] = img[1::2, 0::2, 1]
    new_img[1::2, 1::2, 2] = img[1::2, 1::2, 2]

    return new_img


In [10]:
from scipy import signal, ndimage

def bilinear(img):
    new_img = np.zeros(img.shape).astype(float)

    conv_kernel_1 = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
    conv_kernel_2 = np.array([[0, 1, 0], [1, 4, 1], [0, 1, 0]])

    # # 溢出!
    # new_img = np.zeros_like(img)
    # test = (signal.convolve2d(img[:, :, 0], conv_kernel_1, mode='same'))
    # print(test[1, 1])
    # new_img[:, :, 0] = test
    # print(new_img[1, 1])

    new_img[:, :, 0] = signal.convolve2d(img[:, :, 0], conv_kernel_1, mode='same')
    new_img[:, :, 1] = signal.convolve2d(img[:, :, 1], conv_kernel_2, mode='same')
    new_img[:, :, 2] = signal.convolve2d(img[:, :, 2], conv_kernel_1, mode='same')

    new_img = (new_img / 4).clip(0, 255)
    return new_img.astype(np.uint8)

In [11]:
from scipy import signal, ndimage

def GBTF(img):
    S = img.astype(float)

    # RBG
    S[:, :, [1, 2]] = S[:, :, [2, 1]]
    R, B, G = S[:,:,0], S[:,:,1], S[:,:,2]

    G_Final = np.copy(G)
    C_Final = [np.copy(R), np.copy(B)]

    conv_kernel_1 = [-1, 2, 2, 2, -1]
    H = 1/4 * ndimage.convolve1d((R+G+B), conv_kernel_1)
    G_H = np.copy(G)
    G_H[0::2, 0::2] = H[0::2, 0::2]
    G_H[1::2, 1::2] = H[1::2, 1::2]

    R_H = np.copy(R)
    R_H[0::2, 1::2] = H[0::2, 1::2]
    B_H = np.copy(B)
    B_H[1::2, 0::2] = H[1::2, 0::2]

    V = 1/4 * ndimage.convolve1d((R+G+B).T, conv_kernel_1).T
    G_V = np.copy(G)
    G_V[0::2, 0::2] = V[0::2, 0::2]
    G_V[1::2, 1::2] = V[1::2, 1::2]

    R_V = np.copy(R)
    R_V[1::2, 0::2] = V[1::2, 0::2]
    B_V = np.copy(B)
    B_V[0::2, 1::2] = V[0::2, 1::2]

    delta_H = G_H - R_H - B_H
    delta_V = G_V - R_V - B_V

    conv_kernel_2 = [-1, 0, 1]
    D_H = np.absolute(ndimage.convolve1d(delta_H, conv_kernel_2))
    D_V = np.absolute(ndimage.convolve1d(delta_V.T, conv_kernel_2)).T

    conv_kernel_3 = np.array([[0 for i in range(9)] for j in range(9)])

    conv_kernel_3[:, :] = 0
    conv_kernel_3[2:7, 0:5] = 1
    W_W = signal.convolve2d(D_H, conv_kernel_3, mode='same')

    conv_kernel_3[:, :] = 0
    conv_kernel_3[2:7, 4:9] = 1
    W_E = signal.convolve2d(D_H, conv_kernel_3, mode='same')

    conv_kernel_3[:,  :] = 0
    conv_kernel_3[0:5, 2:7] = 1
    W_N = signal.convolve2d(D_V, conv_kernel_3, mode='same')

    conv_kernel_3[:,  :] = 0
    conv_kernel_3[4:9, 2:7] = 1
    W_S = signal.convolve2d(D_V, conv_kernel_3, mode='same')

    # 如果某个方向是零，表示这一大片都是一个颜色，此时需要直接根据该方向的结果来作为最后的预测值
    # 因此，设定很小的一个值，取倒数后很大，之后计算的时候，其他几个方向都像被忽略不算一样，从而实现目标
    W_W[W_W==0] = 1e-10
    W_E[W_E==0] = 1e-10
    W_N[W_N==0] = 1e-10
    W_S[W_S==0] = 1e-10

    W_W = 1 / np.square(W_W)
    W_E = 1 / np.square(W_E)
    W_N = 1 / np.square(W_N)
    W_S = 1 / np.square(W_S)

    f = [1, 2, 3, 4, 5]
    conv_kernel_4 = np.array([0 for i in range(9)])

    delta_Final = list()
    for c in range(2):
        now_delta_V = np.copy(delta_V)
        now_delta_H = np.copy(delta_H)

        now_delta_H[c::2, :] = delta_H[c::2, :]
        now_delta_V[:, c::2] = delta_V[:, c::2]

        conv_kernel_4[:] = 0
        conv_kernel_4[0:5] = f
        V1 = ndimage.convolve1d(now_delta_V.T, conv_kernel_4).T
        conv_kernel_4[:] = 0
        conv_kernel_4[4:9] = f[::-1]
        V2 = ndimage.convolve1d(now_delta_V.T, conv_kernel_4).T
        conv_kernel_4[:] = 0
        conv_kernel_4[0:5] = f
        V3 = ndimage.convolve1d(now_delta_H, conv_kernel_4)
        conv_kernel_4[:] = 0
        conv_kernel_4[4:9] = f[::-1]
        V4 = ndimage.convolve1d(now_delta_H, conv_kernel_4)

        delta_Final.append(1/np.sum(f) * (V1 * W_N + V2 * W_S + V3 * W_E + V4 * W_W) / (W_N + W_S + W_E + W_W))

    [delta_R, delta_B] = delta_Final

    G_Final[0::2, 0::2] = R[0::2, 0::2] + delta_R[0::2, 0::2]
    G_Final[1::2, 1::2] = B[1::2, 1::2] + delta_B[1::2, 1::2]

    conv_kernel_5 = np.array([[0 for _ in range(7)] for _ in range(7)])
    conv_kernel_5[0, 2] = conv_kernel_5[0, 4] = conv_kernel_5[2, 0] = conv_kernel_5[2, 6] = -1
    conv_kernel_5[6, 2] = conv_kernel_5[6, 4] = conv_kernel_5[4, 0] = conv_kernel_5[4, 6] = -1
    conv_kernel_5[2, 2] = conv_kernel_5[2, 4] = conv_kernel_5[4, 2] = conv_kernel_5[4, 4] = 10

    conv_kernel_6 = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
    for c in range(2):
        C_adjust = G_Final - signal.convolve2d(delta_Final[c], conv_kernel_5, mode='same') / 32
        C_Final[c][1-c::2, 1-c::2] = C_adjust[1-c::2, 1-c::2]

        C_adjust = G - signal.convolve2d(G_Final - C_Final[c], conv_kernel_6, mode='same') / 4

        C_Final[c][c::2, 1-c::2] = C_adjust[c::2, 1-c::2]
        C_Final[c][1-c::2, c::2] = C_adjust[1-c::2, c::2]

    new_img = np.dstack((C_Final[0], G_Final, C_Final[1]))
    new_img = (new_img + 0.5).clip(0, 255)

    return new_img.astype(np.uint8)

In [12]:
from scipy import signal, ndimage

def RI(img):
    def guide_filter(M, I, p, h, v, eps=0.01):
        sum_kernel = np.ones((h, v))

        N1 = signal.convolve2d(M, sum_kernel, mode='same')
        N2 = signal.convolve2d(np.ones(I.shape), sum_kernel, mode='same')

        nI = I * M
        mean_I = signal.convolve2d(nI, sum_kernel, mode='same') / N1
        mean_p = signal.convolve2d(p, sum_kernel, mode='same') / N1
        corr_I = signal.convolve2d(nI*nI, sum_kernel, mode='same') / N1
        corr_Ip = signal.convolve2d(nI*p, sum_kernel, mode='same') / N1

        var_I = corr_I - mean_I * mean_I
        cov_Ip = corr_Ip - mean_I * mean_p

        a = cov_Ip / (var_I + eps)
        b = mean_p - a * mean_I

        mean_a = signal.convolve2d(a, sum_kernel, mode='same') / N2
        mean_b = signal.convolve2d(b, sum_kernel, mode='same') / N2

        return mean_a * I + mean_b

    # Step 1: Use GBTF to infer G
    G = GBTF(img)[:, :, 1].astype(float)
    G = (G + 0.5).clip(0, 255)

    R = img[:, :, 0].astype(float)
    B = img[:, :, 2].astype(float)


    # Step 2: Use guide filter to predict R or B
    R_Mask = np.zeros(G.shape)
    R_Mask[0::2, 0::2] = 1
    new_R = guide_filter(R_Mask, G, R, 11, 11)

    B_Mask = np.zeros(G.shape)
    B_Mask[1::2, 1::2] = 1
    new_B = guide_filter(B_Mask, G, B, 11, 11)

    # Step 3: Calculate delta of the positions which had data before
    delta_R = (R - new_R) * R_Mask
    delta_B = (B - new_B) * B_Mask

    # Step 4: Interploate the RI
    intp = np.array([[1/4, 1/2, 1/4], [1/2, 1, 1/2], [1/4, 1/2, 1/4]])
    delta_R = signal.convolve2d(delta_R, intp, mode='same') / signal.convolve2d(R_Mask, intp, mode='same')
    delta_B = signal.convolve2d(delta_B, intp, mode='same') / signal.convolve2d(B_Mask, intp, mode='same')

    # Step 5: Add, done, nice job!
    new_R = new_R + delta_R
    new_B = new_B + delta_B

    new_R[0::2, 0::2] = R[0::2, 0::2]
    new_B[1::2, 1::2] = B[1::2, 1::2]

    new_R = (new_R + 0.5).clip(0, 255)
    new_B = (new_B + 0.5).clip(0, 255)

    new_img = np.dstack((new_R, G, new_B))
    return new_img.astype(np.uint8)

In [ ]:
import cv2

for picname in ['./kodim19.png']:
    src_img = np.array(Image.open(picname))

    bayer_img = make_bayer(src_img)
    display(Image.fromarray(bayer_img))

    bilinear_img = bilinear(bayer_img)
    display(Image.fromarray(bilinear_img))
    print('Bilinear: ', metrics(src_img, bilinear_img))

    gbtf_img = GBTF(bayer_img)
    display(Image.fromarray(gbtf_img))
    print('GBTF: ', metrics(src_img, gbtf_img))

    ri_img = RI(bayer_img)
    display(Image.fromarray(ri_img))
    print('RI: ', metrics(src_img, ri_img))